### <p style="color:yellow;">Descrição do problema:</p>

Dados os obstáculos enfrentados pelo sistema educacional, a evasão escolar
desempenha um papel crucial, afetando estudantes de diferentes contextos
socioeconômicos. Com base nisso, este trabalho propõe **construir um modelo
preditivo baseado em técnicas de Ciência de Dados para prever a taxa de evasão no
ensino médio e técnico brasileiro**. O estudo aborda a complexidade deste fenômeno,
destacando a sua natureza social e a necessidade de soluções proativas. A
abordagem utiliza dados do censo escolar da educação básica e inclui a análise
exploratória de dados e técnicas de aprendizado de máquina. O modelo proposto visa
prever a evasão escolar, permitindo uma intervenção personalizada e prestando
apoio específico aos alunos desfavorecidos. Além de contribuir para a compreensão
dos fatores associados à evasão escolar, os resultados pretendem oferecer uma
abordagem inovadora para a prevenção desse fenômeno, indo além do meio
acadêmico de modo a proporcionar contribuições práticas ao campo educacional.

### <p style="color:yellow;">Algoritmos a serem utilizados:</p>

- **Random Forest**:

    - Motivo: Por se tratar de um algoritmo baseado em árvores de decisão, Random Forest pode ser uma solução robusta
    e capaz de lidar com diferentes características no conjunto de dados.

- **Gradient Boosting Machines (GBM)**:

    - Motivo: Assim como o Random Forest, o GBM é um algoritmo baseado em árvores, porém constrói modelos de maneira
    sequencial, onde cada novo modelo subsequente tende a corrigir as previsões dos modelos anteriores. É uma solução
    eficaz para capturar padrões complexos.

- **Support Vector Machine (SVM)**:

    - Motivo: O SVM é útil em situações onde a separação entre classes não é linear, onde este algoritmo irá projetar 
    os dados em um espaço dimensional maior onde a separação é viável. Utilizar o SVM pode ser uma ótima solução para
    encontrar uma fronteira de decisão ótima entre alunos que evadem e alunos que não evadem.

- **Logistic Regression**:

    - Motivo: É um modelo de fácil interpretação e bastante robusto e eficaz quando se trata de problemas de classificação ou quando há uma relação linear entre as carcterísticas e a probabilidade de evasão.

- **K-Nearest Neighbors (KNN)**:

    - Motivo: O KNN é um algoritmo que apesar de simples, pode ser útil para captar grupos similares
    entre os alunos, onde utiliza cálculos de distância para encontrar os vizinhos mais próximos a um
    ponto específico, atribuindo uma nova classe.


### Importação das bibliotecas

In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt


#from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsClassifier
#from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.model_selection import  GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import OneHotEncoder,  MinMaxScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy import stats
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import accuracy_score,  precision_score, f1_score, recall_score,classification_report

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import inflection
import warnings
warnings.filterwarnings("ignore")

In [2]:
dados_tec = pd.read_csv('../data/processed/preprocessing.csv')
dados_tec.head()

,Carga Horária,Categoria da Situação,Código da Matrícula,Cor/Raça,Data de Início do Ciclo,Data de Fim Previsto do Ciclo,Faixa Etária,Fator Esforço Curso,Idade,Instituição,Mês de Ocorrência da Situação,Região,Renda Familiar,Sexo,Turno,UF,Unidade de Ensino
0,4560,Concluintes,72082427,branca,2017-04-07,2019-12-21,15 a 19 anos,1.0,18.0,CPII,2019-12-01,Região Sudeste,"1,5<RFP<=2,5",Feminino,Matutino,RJ,Campus Tijuca II
1,4560,Concluintes,72082311,branca,2017-04-07,2019-12-21,15 a 19 anos,1.0,18.0,CPII,2019-12-01,Região Sudeste,Não declarada,Masculino,Matutino,RJ,Campus Tijuca II
2,4560,Concluintes,72082437,branca,2017-04-07,2019-12-21,15 a 19 anos,1.0,18.0,CPII,2019-12-01,Região Sudeste,"RFP>3,5",Feminino,Matutino,RJ,Campus Tijuca II
3,4560,Concluintes,72898296,branca,2017-04-07,2019-12-21,15 a 19 anos,1.0,18.0,CPII,2019-12-01,Região Sudeste,Não declarada,Feminino,Matutino,RJ,Campus Tijuca II
4,4560,Concluintes,72082281,branca,2017-04-07,2019-12-21,15 a 19 anos,1.0,18.0,CPII,2019-12-01,Região Sudeste,"RFP>3,5",Feminino,Matutino,RJ,Campus Tijuca II


In [3]:
dados_tec.shape

(4250, 17)

### Renomeando colunas

In [4]:
dados_tec.columns = dados_tec.columns.str.replace(' ','')
new_cols = []
for column in dados_tec.columns:
    new_cols.append(inflection.underscore(column))
dados_tec.columns = new_cols

In [5]:
dados_tec.rename(columns={'categoriada_situação':'categoria_da_situação',
                            'datade_iníciodo_ciclo':'data_de_inicio_do_ciclo',
                            'datade_fim_previstodo_ciclo':'data_de_fim_previsto_do_ciclo',
                            'mêsde_ocorrênciada_situação':'mês_de_ocorrência_da_situação'},inplace=True)

In [6]:
dados_tec.columns

Index(['carga_horária', 'categoria_da_situação', 'códigoda_matrícula',
       'cor/raça', 'data_de_inicio_do_ciclo', 'data_de_fim_previsto_do_ciclo',
       'faixa_etária', 'fator_esforço_curso', 'idade', 'instituição',
       'mês_de_ocorrência_da_situação', 'região', 'renda_familiar', 'sexo',
       'turno', 'uf', 'unidadede_ensino'],
      dtype='object')

### Filtragem de variáveis

Existem algumas varíaveis que foram necessárias apenas para derivar outras e que não faz mais sentido mantê-las. São:
- data_de_inicio_do_ciclo
- data_de_fim_previsto_do_ciclo
- mês_de_ocorrência_da_situação

In [7]:
dados_tec.drop(columns=['data_de_inicio_do_ciclo','data_de_fim_previsto_do_ciclo',
                          'mês_de_ocorrência_da_situação'],inplace=True)

### Separação de Features e Target

In [8]:
features_tec = dados_tec.drop(columns=['categoria_da_situação'])
features_tec.head(3)

,carga_horária,códigoda_matrícula,cor/raça,faixa_etária,fator_esforço_curso,idade,instituição,região,renda_familiar,sexo,turno,uf,unidadede_ensino
0,4560,72082427,branca,15 a 19 anos,1.0,18.0,CPII,Região Sudeste,"1,5<RFP<=2,5",Feminino,Matutino,RJ,Campus Tijuca II
1,4560,72082311,branca,15 a 19 anos,1.0,18.0,CPII,Região Sudeste,Não declarada,Masculino,Matutino,RJ,Campus Tijuca II
2,4560,72082437,branca,15 a 19 anos,1.0,18.0,CPII,Região Sudeste,"RFP>3,5",Feminino,Matutino,RJ,Campus Tijuca II


In [9]:
target_tec = dados_tec[['categoria_da_situação']]

### Separação de treino e teste

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features_tec, target_tec, test_size=0.2)

In [11]:
print(f'X_train: {X_train.shape[0]} amostras')
print(f'X_test: {X_test.shape[0]} amostras')

X_train: 3400 amostras
X_test: 850 amostras


### Pré-processamento das variáveis

Nesta etapa iremos definir os transformadores para codificar as variáveis categóricas, bem como escalonar as variáveis numéricas.

In [12]:
categorical_features = X_train.select_dtypes(include=['object']).columns
numerical_features = X_train.select_dtypes(exclude=['object']).columns

In [13]:
encoder = OneHotEncoder(handle_unknown='ignore',drop='first')
minmax_scaler = MinMaxScaler()

In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', minmax_scaler, numerical_features),
        ('categorical', encoder, categorical_features)
])

### Definição dos modelos

In [15]:
models = [
    {
        "model_name": "Logistic Regression",
        "model": LogisticRegression(solver="liblinear", max_iter=1000),
        "model_params": {"classifier__penalty": ["l1", "l2"], "classifier__C": [0.1,1.2]},
    },
    {
        "model_name": "KNN",
        "model": KNeighborsClassifier(metric="euclidean"),
        "model_params": {
            "classifier__n_neighbors": np.arange(1, 20, 2),
            "classifier__weights": ["uniform", "distance"],
        },
    },
    {
        "model_name": "Random Forest",
        "model": RandomForestClassifier(),
        "model_params": {
            "classifier__criterion": ["gini", "entropy"],
            "classifier__n_estimators": [100, 200, 300],
            "classifier__max_depth": [None, 10, 20, 30],
            "classifier__min_samples_split": [2, 5, 10],
            "classifier__min_samples_leaf": [1, 2, 4],
        },
    },
    {
        "model_name": "GBM",
        "model": GradientBoostingClassifier(),
        "model_params": {
            "classifier__criterion": ["friedman_mse", "squared_error"],
            "classifier__n_estimators": [100, 200, 300],
            "classifier__learning_rate": [0.01, 0.1, 0.2],
            "classifier__max_depth": [3, 5, 7],
            "classifier__subsample": [0.8, 0.9, 1.0],
            "classifier__min_samples_split": [2, 5, 10],
        },
    },
    {
        "model_name": "SVM",
        "model": SVC(),
        "model_params": {
            "classifier__C": [0.1, 1, 10, 100],
            "classifier__gamma": [1, 0.1, 0.01, 0.001],
            "classifier__kernel": ["rbf", "linear", "poly"],
        },
    },
]

### Cross validation e Fine tuning

In [16]:
results = pd.DataFrame({})

In [17]:
for model in models:

    models_results = {}

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model['model'])
    ])
    # Inner CV
    grid_search = GridSearchCV(pipeline, model['model_params'],cv=5, scoring='accuracy')
    # Outer CV
    cross_validation_results = cross_validate(grid_search, X=X_train, y=y_train,
                             cv=5, scoring=['accuracy', 'f1', 'precision', 'recall'])

    acc_scores = f"{cross_validation_results['test_accuracy'].mean():.3f} +- {cross_validation_results['test_accuracy'].std():.3f}"
    f1_scores = f"{cross_validation_results['test_f1'].mean():.3f} +- {cross_validation_results['test_f1'].std():.3f}"
    precision_scores = f"{cross_validation_results['test_precision'].mean():.3f} +- {cross_validation_results['test_precision'].std():.3f}"
    recall_scores = f"{cross_validation_results['test_recall'].mean():.3f} +- {cross_validation_results['test_recall'].std():.3f}"

    models_results['model_name'] = [model['model_name']]
    models_results['accuracy'] = [acc_scores]
    models_results['f1'] = [f1_scores]
    models_results['precision'] = [precision_scores]
    models_results['recall'] = [recall_scores]

    df = pd.DataFrame(models_results)
    results = pd.concat([results, df])

In [ ]:
# for model in models:
#     print(model['model'])
    
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('classifier model', model['model'])
#     ])
#     # Inner CV
#     grid_search = GridSearchCV(pipeline, model['model_params']cv=5, n_jobs=-1, 
#                                scoring='accuracy')
#     # Outer CV
#     results = cross_validate(grid_search, X=X_train, y=y_train,
#                              cv=5, scoring=['accuracy', 'f1', 'precision', 'recall'])
    